In [29]:
import pandas as pd
from io import StringIO
import os
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Union, Set
from enum import Enum

from nst_scraper import nst_on_ice_scraper
from db_utils import insert_player, update_player_db, check_last_update, append_player_ids
from team_utils import get_most_recent_game_id

from dotenv import load_dotenv

pd.set_option('display.max_columns', None)


In [30]:
player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='')
player_stats_df.head()


,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,STL,D,20,388.516667,1,3,1,2,4,20.00,17,5.88,0.77,50,28,11,2,3,5,12,6,6,0,0,1,22,4,11,14,22,0,0,-
1,Brent Burns,CAR,D,18,294.850000,0,5,2,3,5,29.41,29,0.00,1.18,75,46,20,2,1,6,4,2,2,0,0,1,25,8,2,10,14,0,0,-
2,Corey Perry,EDM,R,20,204.600000,3,0,0,0,3,42.86,18,16.67,2.20,38,26,22,11,0,1,14,4,2,2,0,7,6,3,5,10,5,0,2,0.00
3,Alex Ovechkin,WSH,L,18,215.250000,9,7,4,3,16,72.73,34,26.47,3.27,69,49,40,16,0,5,6,3,3,0,0,0,7,4,41,8,3,0,0,-
4,Evgeni Malkin,PIT,C,21,316.166667,3,9,6,3,12,75.00,30,10.00,4.62,61,40,36,20,0,8,8,4,4,0,0,1,20,3,9,18,15,62,80,43.66


In [31]:
goalie_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='', pos='g')
goalie_stats_df.head()


,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,4,198.850000,92,85,7,0.924,2.11,1.05,5.51,12,9,3,0.750,0.91,-0.83,23,19,4,0.826,1.21,-1.40,51,51,0,1.000,0.00,1.71,6,10,39.33,20.14
1,Jonathan Quick,NYR,5,225.233333,109,108,1,0.991,0.27,8.54,8.91,27,27,0,1.000,0.00,4.89,32,31,1,0.969,0.27,2.62,47,47,0,1.000,0.00,1.57,11,21,34.87,30.00
2,James Reimer,ANA,2,103.600000,53,48,5,0.906,2.90,-0.36,5.20,7,4,3,0.571,1.74,-1.73,23,21,2,0.913,1.16,0.60,23,23,0,1.000,0.00,0.77,3,8,30.81,17.20
3,Semyon Varlamov,NYI,8,405.900000,174,160,14,0.920,2.07,1.23,15.72,49,43,6,0.878,0.89,2.87,42,37,5,0.881,0.74,-0.25,67,64,3,0.955,0.44,-0.76,14,28,35.54,20.21
4,Jacob Markstrom,N.J,14,683.216667,317,288,29,0.909,2.55,-1.26,26.59,71,54,17,0.761,1.49,-4.15,88,80,8,0.909,0.70,1.95,138,134,4,0.971,0.35,0.62,41,59,39.39,21.07


In [32]:
class Position(Enum):
    C = 'C'
    L = 'L'
    R = 'R'
    D = 'D'
    G = 'G'
    
    @property
    def category(self) -> str:
        if self in {Position.C, Position.L, Position.R}:
            return 'F'
        elif self == Position.D:
            return 'D'
        elif self == Position.G:
            return 'G'
    
    def __str__(self) -> str:
        return self.value

In [33]:
@dataclass
class Player:
    name: str
    team: str
    position: Position
    player_id: Optional[int] = None

    def __str__(self) -> str:
        """
        Returns a string representation of the player.
        """
        return f"{self.name} ({self.position}) - {self.team}"

    def to_dict(self) -> Dict[str, Optional[str]]:
        """
        Converts the Player instance into a dictionary.
        
        Returns:
            Dict[str, Optional[str]]: A dictionary representation of the player.
        """
        return {
            'player_id': self.player_id,
            'name': self.name,
            'team': self.team,
            'position': self.position.value
        }

In [34]:
@dataclass
class Lineup:
    name: str
    forwards: List[Optional[Player]] = field(default_factory=lambda: [None] * 12)
    defense: List[Optional[Player]] = field(default_factory=lambda: [None] * 6)
    goalies: List[Optional[Player]] = field(default_factory=lambda: [None] * 2)
    
    ALLOWED_FORWARD_CATEGORIES = {'F'}
    ALLOWED_DEFENSE_CATEGORY = 'D'
    ALLOWED_GOALIE_CATEGORY = 'G'
    
    def __post_init__(self):
        self.validate_lineup_size()
    
    def validate_lineup_size(self):
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if total_players > 20:
            raise ValueError(f"Total number of players ({total_players}) exceeds the hard limit of 20.")
    
    def add_player(
        self,
        category: str,
        player: Player,
        slot: int,
        allowed_categories: Union[str, Set[str]],
        max_slots: int
    ):
        """
        Adds a player to the specified category and slot after validating their position category.
        Ensures that the total number of players does not exceed 20.
        """
        if isinstance(allowed_categories, str):
            allowed_categories = {allowed_categories}
        elif isinstance(allowed_categories, set):
            allowed_categories = allowed_categories
        else:
            raise TypeError("allowed_categories must be a string or a set of strings.")
        
        if player.position.category not in allowed_categories:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position.value}' "
                f"to {category}. Allowed categories: {', '.join(allowed_categories)}."
            )
        
        if not 0 <= slot < max_slots:
            raise IndexError(f"{category.capitalize()} slot must be between 0 and {max_slots - 1}.")
        
        current_category = getattr(self, category)
        if current_category[slot]:
            existing_player = current_category[slot].name
            print(f"Warning: Slot {slot + 1} in {category} is already occupied by '{existing_player}'. Overwriting.")
        
        # Check total players before adding
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if current_category[slot] is None and total_players >= 20:
            raise ValueError("Cannot add more players. The lineup has reached the hard limit of 20 players.")
        
        current_category[slot] = player
        setattr(self, category, current_category)
        print(f"Added player '{player.name}' to {category[:-1].capitalize()} slot {slot + 1}.")
    
    def add_forward(self, player: Player, slot: int):
        self.add_player(
            category='forwards',
            player=player,
            slot=slot,
            allowed_categories=self.ALLOWED_FORWARD_CATEGORIES,
            max_slots=len(self.forwards)
        )
    
    def add_defense(self, player: Player, slot: int):
        self.add_player(
            category='defense',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_DEFENSE_CATEGORY},
            max_slots=len(self.defense)
        )
    
    def set_goalie(self, player: Player, slot: int):
        self.add_player(
            category='goalies',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_GOALIE_CATEGORY},
            max_slots=len(self.goalies)
        )
    
    def adjust_slots(self, category: str, delta: int):
        """
        Adjusts the number of slots in the specified category by delta.
        Allows ±1 adjustment only.
        
        Args:
            category (str): The category to adjust ('forwards' or 'defense').
            delta (int): The change in number of slots (+1 or -1).
        """
        if category not in {'forwards', 'defense'}:
            raise ValueError("Can only adjust 'forwards' or 'defense' categories.")
        if delta not in {-1, 1}:
            raise ValueError("Delta must be either +1 or -1.")
        
        current_slots = getattr(self, category)
        new_slot_count = len(current_slots) + delta
        
        if category == 'forwards':
            if not (11 <= new_slot_count <= 13):
                raise ValueError("Number of forwards can only vary by ±1 from the default of 12.")
        elif category == 'defense':
            if not (5 <= new_slot_count <= 7):
                raise ValueError("Number of defensemen can only vary by ±1 from the default of 6.")
        
        if delta == 1:
            current_slots.append(None)
        elif delta == -1:
            removed_player = current_slots.pop()
            if removed_player:
                print(f"Removed player '{removed_player.name}' from {category}.")
        
        setattr(self, category, current_slots)
        print(f"Adjusted {category} slots to {len(getattr(self, category))}.")
        self.validate_lineup_size()
    
    def display_lineup(self):
        """
        Prints the current lineup in a structured format.
        """
        print(f"Lineup: {self.name}\n")
        
        for category, title in [('forwards', 'Forwards'), ('defense', 'Defense'), ('goalies', 'Goalies')]:
            print(f"{title}:")
            for idx, player in enumerate(getattr(self, category), start=1):
                player_info = str(player) if player else 'Empty'
                print(f"  Slot {idx}: {player_info}")
            print()
    
    def to_dataframe(self) -> pd.DataFrame:
        """
        Converts the lineup into a pandas DataFrame.
        Conditionally includes player attributes if they are present.
        """
        data = []
        for category, pos in [('forwards', 'f'), ('defense', 'd'), ('goalies', 'g')]:
            for idx, player in enumerate(getattr(self, category), start=1):
                player_dict = {
                    'Position': f"{pos}{idx}",
                    'Player': player.name if player else 'Empty'
                }
                # Conditionally add 'player_id' if it exists
                if player and player.player_id is not None:
                    player_dict['Player ID'] = player.player_id
                data.append(player_dict)
        
        df = pd.DataFrame(data)
        
        # Optionally, remove columns where all values are NaN
        df.dropna(axis=1, how='all', inplace=True)
        
        return df
    
    def to_transposed_dataframe(self) -> pd.DataFrame:
        """
        Transposes the lineup DataFrame so that each column represents a position-slot combination
        and the row contains the corresponding player names.
        Conditionally includes additional player attributes if they are present.
        """
        df = self.to_dataframe()
        
        # Initialize dictionaries to hold player names and optional IDs
        player_data = {}
        id_data = {}
        
        for _, row in df.iterrows():
            pos = row['Position']
            player_name = row['Player']
            player_data[pos] = player_name
            
            # Handle 'Player ID' if it exists
            if 'Player ID' in row and pd.notna(row['Player ID']):
                id_data[f"{pos}_ID"] = row['Player ID']
        
        # Combine player names and IDs into a single dictionary
        transposed_data = {**player_data, **id_data}
        
        # Create the transposed DataFrame with a single row
        transposed_df = pd.DataFrame([transposed_data])
        
        return transposed_df

In [35]:
# Creating Player instances from the player_stats_df DataFrame
player_list = []
for _, row in player_stats_df.iterrows():
    try:
        position_enum = Position(row['position'])  # Convert abbreviation to Position Enum
    except ValueError:
        print(f"Invalid position '{row['position']}' for player '{row['player']}'. Skipping.")
        continue
    
    player = Player(
        name=row['player'],
        team=row['team'],
        position=position_enum
        # player_id is not set initially
    )
    player_list.append(player)

# Convert player list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in player_list]).head()

Invalid position 'C, R' for player 'Gustav Nyquist'. Skipping.
Invalid position 'C, L' for player 'Jeff Skinner'. Skipping.
Invalid position 'C, L' for player 'Mark Jankowski'. Skipping.
Invalid position 'C, R' for player 'Michael McCarron'. Skipping.
Invalid position 'C, R' for player 'Philip Tomasino'. Skipping.
Invalid position 'L, R' for player 'Cole Smith'. Skipping.


,name,team,position
0,Ryan Suter,STL,D
1,Brent Burns,CAR,D
2,Corey Perry,EDM,R
3,Alex Ovechkin,WSH,L
4,Evgeni Malkin,PIT,C


In [36]:
goalie_stats_df.head()

,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,4,198.850000,92,85,7,0.924,2.11,1.05,5.51,12,9,3,0.750,0.91,-0.83,23,19,4,0.826,1.21,-1.40,51,51,0,1.000,0.00,1.71,6,10,39.33,20.14
1,Jonathan Quick,NYR,5,225.233333,109,108,1,0.991,0.27,8.54,8.91,27,27,0,1.000,0.00,4.89,32,31,1,0.969,0.27,2.62,47,47,0,1.000,0.00,1.57,11,21,34.87,30.00
2,James Reimer,ANA,2,103.600000,53,48,5,0.906,2.90,-0.36,5.20,7,4,3,0.571,1.74,-1.73,23,21,2,0.913,1.16,0.60,23,23,0,1.000,0.00,0.77,3,8,30.81,17.20
3,Semyon Varlamov,NYI,8,405.900000,174,160,14,0.920,2.07,1.23,15.72,49,43,6,0.878,0.89,2.87,42,37,5,0.881,0.74,-0.25,67,64,3,0.955,0.44,-0.76,14,28,35.54,20.21
4,Jacob Markstrom,N.J,14,683.216667,317,288,29,0.909,2.55,-1.26,26.59,71,54,17,0.761,1.49,-4.15,88,80,8,0.909,0.70,1.95,138,134,4,0.971,0.35,0.62,41,59,39.39,21.07


In [37]:
# Creating Player instances from the goalie_stats_df DataFrame
goalie_list = []
for _, row in goalie_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position=Position.G  # Set position to 'G' for goalies
        # player_id is not set initially
    )
    goalie_list.append(player)

# Convert goalie list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in goalie_list]).head()

,name,team,position
0,Marc-Andre Fleury,MIN,G
1,Jonathan Quick,NYR,G
2,James Reimer,ANA,G
3,Semyon Varlamov,NYI,G
4,Jacob Markstrom,N.J,G


In [38]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2024-11-20'

In [39]:
# Load environment variables from .env file
load_dotenv()

# Construct the database configuration dictionary
db_config = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

In [40]:
# Check the last update time of the players database
last_update = check_last_update(db_config)

INFO:db_utils:Database connection established.
INFO:db_utils:Last database update was on: 2024-11-21
INFO:db_utils:Database connection closed.


In [41]:
# Convert last_update to datetime
last_update_dt = datetime.strptime(last_update, '%Y-%m-%d')
today_dt = datetime.strptime(today_datetime.strftime('%Y-%m-%d'), '%Y-%m-%d')
yesterday_dt = datetime.strptime(yesterday, '%Y-%m-%d')

# Only update if last update was before yesterday
if last_update_dt not in [today_dt, yesterday_dt]:
    # Update the player database from last update to yesterday
    update_player_db(last_update, yesterday, db_config, skip_existing=True)
else:
    print(f"No need to update the player database. Last update was on: {last_update}")


No need to update the player database. Last update was on: 2024-11-21


In [42]:
def add_player_to_lineup(lineup: Lineup, player: Player, category: str):
    """
    Adds a player to the lineup in the specified category, handling potential errors.
    
    Args:
        lineup (Lineup): The lineup object.
        player (Player): The player to add.
        category (str): The category ('forwards', 'defense', 'goalies').
    """
    try:
        if category == 'forwards':
            slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
            lineup.add_forward(player, slot)
        elif category == 'defense':
            slot = next(i for i, p in enumerate(lineup.defense) if p is None)
            lineup.add_defense(player, slot)
        elif category == 'goalies':
            slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, slot)
        else:
            print(f"Unknown category '{category}'.")
    except StopIteration:
        print(f"No available slots to add player '{player.name}' in category '{category}'.")
    except ValueError as ve:
        print(ve)
    except IndexError as ie:
        print(ie)

In [43]:
# TODO this function just creates a lineup from the player_list and goalie_list
def create_lineup(team) -> Lineup:
    """
    Creates and displays a lineup consisting of players from the specified team.
    
    Args:
        team (str): The team name to filter players.
    """
    # Creating two lineup objects
    lineup1 = Lineup("Lineup 1")
    
    # Adding forwards to lineup1
    forward_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_forward(player, forward_count)
                forward_count += 1
                if forward_count >= 12:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding defense to lineup1
    defense_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_defense(player, defense_count)
                defense_count += 1
                if defense_count >= 6:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding goalies to lineup1
    goalie_count = 0
    for goalie in goalie_list:
        if goalie.team != team:
            continue  # Proceed to the next goalie if not in the specified team
        if goalie_count >= 2:
            print("Maximum of two goalies have been assigned.")
            break
        try:
            lineup1.set_goalie(goalie, goalie_count)
            goalie_count += 1
        except ValueError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
        except IndexError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
    
    # Display the lineup
    # lineup1.display_lineup()
    return lineup1

my_lineup = create_lineup('TOR')

Added player 'Ryan Reaves' to Forward slot 1.
Added player 'Max Pacioretty' to Forward slot 2.
Added player 'John Tavares' to Forward slot 3.
Skipping player 'Oliver Ekman-Larsson': Cannot add player 'Oliver Ekman-Larsson' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Chris Tanev': Cannot add player 'Chris Tanev' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Jani Hakanpää': Cannot add player 'Jani Hakanpää' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Morgan Rielly': Cannot add player 'Morgan Rielly' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Jake McCabe': Cannot add player 'Jake McCabe' with position 'D' to forwards. Allowed categories: F.
Added player 'Max Domi' to Forward slot 4.
Added player 'William Nylander' to Forward slot 5.
Added player 'Mitch Marner' to Forward slot 6.
Added player 'Steven Lorentz' to Forward slot 7.
Skipping player 'Philippe Myers': Cannot add player 'Phil

In [44]:
def create_flexible_lineup(team: str) -> Lineup:
    """
    Creates a flexible lineup for the specified team, allowing for ±1 forward or defense spot.
    
    Args:
        team (str): The team name to filter players.
    
    Returns:
        Lineup: The configured lineup object.
    """
    lineup = Lineup(f"Flexible Lineup for {team}")
    
    # Example logic to adjust slots based on team strategy
    # Here, we simply allow flexibility; implement specific rules as needed
    allow_extra_forward = True  # Example condition
    allow_extra_defense = False  # Example condition
    
    if allow_extra_forward:
        lineup.adjust_slots('forwards', 1)  # Increase forwards to 13
    elif allow_extra_defense:
        lineup.adjust_slots('defense', 1)  # Increase defense to 7
    
    # Add players to the lineup
    forward_count = 0
    defense_count = 0
    goalie_count = 0
    
    for player in player_list:
        if player.team == team:
            if player.position.category == 'F' and forward_count < len(lineup.forwards):
                try:
                    lineup.add_forward(player, forward_count)
                    forward_count += 1
                except ValueError:
                    continue
            elif player.position.category == 'D' and defense_count < len(lineup.defense):
                try:
                    lineup.add_defense(player, defense_count)
                    defense_count += 1
                except ValueError:
                    continue
    
    for goalie in goalie_list:
        if goalie.team == team and goalie_count < len(lineup.goalies):
            try:
                lineup.set_goalie(goalie, goalie_count)
                goalie_count += 1
            except ValueError:
                continue
    
    return lineup

# Create and display a flexible lineup for 'TOR'
flexible_lineup = create_flexible_lineup('TOR')
flexible_lineup.display_lineup()

Adjusted forwards slots to 13.
Added player 'Ryan Reaves' to Forward slot 1.
Added player 'Max Pacioretty' to Forward slot 2.
Added player 'John Tavares' to Forward slot 3.
Added player 'Oliver Ekman-Larsson' to Defens slot 1.
Added player 'Chris Tanev' to Defens slot 2.
Added player 'Jani Hakanpää' to Defens slot 3.
Added player 'Morgan Rielly' to Defens slot 4.
Added player 'Jake McCabe' to Defens slot 5.
Added player 'Max Domi' to Forward slot 4.
Added player 'William Nylander' to Forward slot 5.
Added player 'Mitch Marner' to Forward slot 6.
Added player 'Steven Lorentz' to Forward slot 7.
Added player 'Philippe Myers' to Defens slot 6.
Added player 'Auston Matthews' to Forward slot 8.
Added player 'David Kampf' to Forward slot 9.
Added player 'Connor Dewar' to Forward slot 10.
Added player 'Pontus Holmberg' to Forward slot 11.
Added player 'Nicholas Robertson' to Forward slot 12.
Added player 'Bobby McMann' to Forward slot 13.
Added player 'Anthony Stolarz' to Goalie slot 1.
Lineu

In [45]:
my_lineup.to_dataframe()

,Position,Player
0,f1,Ryan Reaves
1,f2,Max Pacioretty
2,f3,John Tavares
3,f4,Max Domi
4,f5,William Nylander
5,f6,Mitch Marner
6,f7,Steven Lorentz
7,f8,Auston Matthews
8,f9,David Kampf
9,f10,Connor Dewar


In [46]:
# Convert the lineup to a transposed DataFrame
transposed_lineup_df = my_lineup.to_transposed_dataframe()

# Display the transposed DataFrame
transposed_lineup_df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,David Kampf,Connor Dewar,Pontus Holmberg,Nicholas Robertson,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll


In [47]:
def assign_player_ids_to_lineup(lineup, db_config):
    """
    Assigns player IDs to each player in the lineup.

    This function performs the following steps:
        1. Extracts all unique player names from the lineup.
        2. Creates Player instances for each player name.
        3. Uses the append_player_ids function to assign player IDs to each Player object.
        4. Updates the lineup with the corresponding player IDs.

    Args:
        lineup (Lineup): The lineup object containing players.
        db_config (dict): Database configuration with keys: dbname, user, password, host, port.

    Returns:
        Lineup: The updated lineup with player IDs assigned.
    """

    # Extract player names from the lineup
    player_list = []
    for position_group in [lineup.forwards, lineup.defense, lineup.goalies]:
        for player in position_group:
            if player.name != 'Empty':
                player_instance = Player(name=player.name, team=None, position=None)  # Team and position can be set if available
                player_list.append(player_instance)

    # Append player IDs using the existing function
    append_player_ids(player_list, db_config)

    # Create a mapping from player name to player_id
    name_to_id = {player.name: player.player_id for player in player_list if player.player_id is not None}

    # Assign player IDs back to the lineup
    for position_group in [lineup.forwards, lineup.defense, lineup.goalies]:
        for player in position_group:
            if player.name != 'Empty':
                player.player_id = name_to_id.get(player.name, None)

    return lineup

In [48]:
# Assuming you have already created `transposed_lineup_df` and `db_config`
my_lineup = assign_player_ids_to_lineup(my_lineup, db_config)

# Display the updated DataFrame
my_lineup.to_dataframe()

INFO:db_utils:Database connection established.
INFO:db_utils:Assigned player_id 8471817 to Ryan Reaves.
INFO:db_utils:Assigned player_id 8474157 to Max Pacioretty.
INFO:db_utils:Assigned player_id 8475166 to John Tavares.
INFO:db_utils:Assigned player_id 8477503 to Max Domi.
INFO:db_utils:Assigned player_id 8477939 to William Nylander.
INFO:db_utils:Assigned player_id 8478483 to Mitch Marner.
INFO:db_utils:Assigned player_id 8478904 to Steven Lorentz.
INFO:db_utils:Assigned player_id 8479318 to Auston Matthews.
INFO:db_utils:Assigned player_id 8480144 to David Kampf.
INFO:db_utils:Assigned player_id 8480980 to Connor Dewar.
INFO:db_utils:Assigned player_id 8480995 to Pontus Holmberg.
INFO:db_utils:Assigned player_id 8481582 to Nicholas Robertson.
INFO:db_utils:Assigned player_id 8475171 to Oliver Ekman-Larsson.
INFO:db_utils:Assigned player_id 8475690 to Chris Tanev.
INFO:db_utils:Assigned player_id 8475825 to Jani Hakanpää.
INFO:db_utils:Assigned player_id 8476853 to Morgan Rielly.
IN

,Position,Player,Player ID
0,f1,Ryan Reaves,8471817
1,f2,Max Pacioretty,8474157
2,f3,John Tavares,8475166
3,f4,Max Domi,8477503
4,f5,William Nylander,8477939
5,f6,Mitch Marner,8478483
6,f7,Steven Lorentz,8478904
7,f8,Auston Matthews,8479318
8,f9,David Kampf,8480144
9,f10,Connor Dewar,8480980


In [49]:
my_lineup.to_transposed_dataframe()


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2,f1_ID,f2_ID,f3_ID,f4_ID,f5_ID,f6_ID,f7_ID,f8_ID,f9_ID,f10_ID,f11_ID,f12_ID,d1_ID,d2_ID,d3_ID,d4_ID,d5_ID,d6_ID,g1_ID,g2_ID
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,David Kampf,Connor Dewar,Pontus Holmberg,Nicholas Robertson,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll,8471817,8474157,8475166,8477503,8477939,8478483,8478904,8479318,8480144,8480980,8480995,8481582,8475171,8475690,8475825,8476853,8476931,8479026,8476932,8479361


In [50]:
def get_player_stats(lineup: Lineup, player_stats_df: pd.DataFrame) -> pd.DataFrame:
    players = [player for player in lineup.forwards + lineup.defense if player]
    player_names = [player.name for player in players]
    return player_stats_df[player_stats_df['player'].isin(player_names)]

lineup_player_stats = get_player_stats(my_lineup, player_stats_df)

In [51]:
lineup_player_stats

,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
9,Ryan Reaves,TOR,R,16,125.083333,0,1,0,1,1,50.00,7,0.00,0.26,13,10,4,0,1,0,13,5,4,1,0,4,4,2,44,7,2,0,0,-
29,Max Pacioretty,TOR,L,13,137.433333,2,4,2,2,6,85.71,21,9.52,3.05,39,34,19,13,1,2,6,3,3,0,0,2,10,2,38,21,5,2,1,66.67
52,John Tavares,TOR,C,19,249.600000,3,7,5,2,10,71.43,34,8.82,4.62,56,47,39,20,3,5,12,6,6,0,0,4,13,5,20,16,9,130,88,59.63
56,Oliver Ekman-Larsson,TOR,D,20,330.316667,1,7,3,4,8,53.33,30,3.33,0.97,66,47,12,3,2,6,20,10,10,0,0,6,22,11,29,17,20,0,0,-
82,Chris Tanev,TOR,D,20,310.866667,0,0,0,0,0,0.00,9,0.00,0.35,31,17,8,2,1,3,10,5,5,0,0,2,17,4,4,38,42,0,0,-
108,Jani Hakanpää,TOR,D,2,26.066667,0,0,0,0,0,0.00,2,0.00,0.03,5,2,1,0,1,0,0,0,0,0,0,0,6,0,1,2,4,0,0,-
154,Morgan Rielly,TOR,D,20,340.683333,2,3,3,0,5,38.46,22,9.09,0.99,53,33,13,1,0,10,4,2,2,0,0,0,22,6,7,35,32,0,0,-
183,Jake McCabe,TOR,D,20,347.666667,0,5,2,3,5,31.25,15,0.00,0.86,53,26,19,3,4,3,15,6,5,1,0,5,17,2,38,38,31,0,0,-
237,Max Domi,TOR,C,19,237.116667,0,5,4,1,5,55.56,14,0.00,0.92,33,22,13,5,1,1,21,9,8,1,0,3,13,9,6,4,11,76,82,48.10
259,William Nylander,TOR,R,20,280.333333,7,4,3,1,11,68.75,51,13.73,4.56,92,66,42,20,5,6,4,2,2,0,0,2,11,3,1,15,5,26,23,53.06


In [52]:
def get_goalie_stats(lineup: Lineup, goalie_stats_df: pd.DataFrame) -> pd.DataFrame:
    goalies = [goalie for goalie in lineup.goalies if goalie]
    goalie_names = [goalie.name for goalie in goalies]
    return goalie_stats_df[goalie_stats_df['player'].isin(goalie_names)]

In [53]:
lineup_goalie_stats = get_goalie_stats(my_lineup, goalie_stats_df)
lineup_goalie_stats

,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
19,Anthony Stolarz,TOR,12,548.883333,267,255,12,0.955,1.31,11.36,22.5,53,46,7,0.868,0.77,2.59,68,63,5,0.926,0.55,2.69,128,128,0,1.000,0.00,4.28,31,53,39.51,18.17
47,Joseph Woll,TOR,6,294.750000,126,117,9,0.929,1.83,2.03,9.2,21,19,2,0.905,0.41,1.80,35,33,2,0.943,0.41,1.96,63,58,5,0.921,1.02,-2.89,9,23,40.42,33.44


In [54]:
yesterday

'2024-11-20'

In [28]:
get_most_recent_game_id('TOR', '2024-11-21')

2024020301

In [55]:
from game_utils import get_game_boxscore, display_boxscore

temp_data = get_game_boxscore(2024020301)

In [56]:
away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(temp_data)
away_skaters


,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8478403,9,J. Eichel,C,0,0,0,-1,0,1,0,3,0.500000,21:31,1,19,1,1,Away
1,8478462,10,N. Roy,C,0,0,0,-1,0,2,0,2,0.166667,19:29,0,20,1,0,Away
2,8481604,16,P. Dorofeyev,L,0,0,0,-1,0,0,0,2,0.000000,19:08,0,18,0,0,Away
3,8479353,21,B. Howden,C,0,0,0,0,2,1,0,0,0.500000,10:50,0,15,0,0,Away
4,8481655,22,C. Schwindt,R,0,0,0,-1,0,1,0,0,0.428571,10:28,0,15,0,0,Away
5,8482125,26,A. Holtz,R,0,0,0,0,2,2,0,0,0.000000,14:43,0,18,0,0,Away
6,8476881,48,T. Hertl,C,0,0,0,-1,0,3,0,2,0.555556,18:03,0,18,1,1,Away
7,8477964,49,I. Barbashev,C,0,0,0,0,0,5,0,2,0.000000,16:27,0,16,2,0,Away
8,8478434,55,K. Kolesar,R,0,0,0,0,0,5,0,2,0.000000,14:03,1,17,1,0,Away
9,8482250,68,C. Burke,C,0,0,0,-1,0,0,0,2,0.000000,09:39,0,13,0,0,Away


In [57]:
away_goalies

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,team
0,8478499,33,A. Hill,G,17/18,3/4,3/3,23/25,0.92,1,1,0,0,2,59:47,True,L,25,23,Away
1,8478492,35,I. Samsonov,G,0/0,0/0,0/0,0/0,NaN,0,0,0,0,0,00:00,False,NaN,0,0,Away


In [58]:
home_skaters

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8478483,16,M. Marner,R,0,2,2,1,0,1,0,1,0.000000,20:45,0,24,3,0,Home
1,8478904,18,S. Lorentz,C,0,0,0,0,0,3,0,2,0.000000,12:54,4,19,0,0,Home
2,8482720,23,M. Knies,L,0,0,0,1,0,0,0,0,0.000000,09:28,0,11,0,0,Home
3,8480980,24,C. Dewar,C,0,0,0,0,0,3,0,1,0.285714,15:56,1,22,0,0,Home
4,8480995,29,P. Holmberg,R,1,0,1,1,0,1,0,1,0.545455,17:40,3,25,2,0,Home
5,8483489,39,F. Minten,C,1,0,1,1,0,2,0,2,0.571429,15:43,0,21,0,0,Home
6,8482634,46,A. Steeves,C,0,0,0,0,0,3,0,1,0.000000,08:29,0,14,0,0,Home
7,8483733,71,N. Grebenkin,R,0,0,0,0,0,4,0,1,0.000000,11:05,1,13,0,0,Home
8,8482259,74,B. McMann,C,0,0,0,0,0,0,0,3,0.000000,16:03,1,20,0,0,Home
9,8477939,88,W. Nylander,R,1,1,2,1,0,0,1,6,0.333333,21:23,1,24,0,1,Home


In [40]:
home_goalies

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,team
0,8476932,41,A. Stolarz,G,24/26,2/3,1/1,27/30,0.9,2,1,0,0,3,60:40,True,W,30,27,Home
1,8479361,60,J. Woll,G,0/0,0/0,0/0,0/0,NaN,0,0,0,0,0,00:00,False,NaN,0,0,Home


In [64]:
from game_utils import get_game_boxscore, display_boxscore
from team_utils import get_most_recent_game_id
from datetime import datetime, timedelta
from typing import Optional

def extract_team_lineup(team: str, reference_date: Optional[str] = None) -> Lineup:
    """
    Extracts the most recent lineup for the specified team based on the latest game data.

    This function performs the following steps:
        1. Determines the reference date (defaults to yesterday if not provided).
        2. Retrieves the most recent game ID for the team using `get_most_recent_game_id`.
        3. Fetches the game boxscore data using `get_game_boxscore`.
        4. Processes the boxscore to obtain skaters and goalies using `display_boxscore`.
        5. Constructs and returns a `Lineup` object populated with the team's players.

    Args:
        team (str): The three-letter team code (e.g., 'TOR').
        reference_date (Optional[str]): The reference date in 'YYYY-MM-DD' format. Defaults to yesterday's date.

    Returns:
        Lineup: A `Lineup` object containing the team's players from the most recent game.

    Raises:
        ValueError: If no recent game is found for the team or if the team is not part of the retrieved game.
    """
    # Step 1: Determine the reference date
    if reference_date is None:
        today_datetime = datetime.now()
        yesterday_datetime = today_datetime - timedelta(days=1, hours=6)  # Adjust for UTC offset if necessary
        reference_date = yesterday_datetime.strftime('%Y-%m-%d')

    # Step 2: Retrieve the most recent game ID for the team
    game_id = get_most_recent_game_id(team, reference_date)
    if game_id is None:
        raise ValueError(f"No recent game found for team '{team}' before {reference_date}.")

    # Step 3: Fetch the game boxscore data
    game_data = get_game_boxscore(game_id)

    # Step 4: Process the boxscore to obtain skaters and goalies
    away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(game_data)

    # Extract team abbrevs to determine if the team is home or away
    away_team_code = game_data.get('awayTeam', {}).get('abbrev')
    home_team_code = game_data.get('homeTeam', {}).get('abbrev')

    if not away_team_code or not home_team_code:
        raise ValueError("Team abbreviations not found in game data.")

    if team.upper() == away_team_code.upper():
        team_side = 'Away'
        skaters = away_skaters
        goalies = away_goalies
    elif team.upper() == home_team_code.upper():
        team_side = 'Home'
        skaters = home_skaters
        goalies = home_goalies
    else:
        raise ValueError(f"Team '{team}' not found in game ID {game_id}.")

    # Step 5: Construct the Lineup object
    lineup = Lineup(name=f"{team.upper()} Lineup from Game {game_id}")

    # Add Skaters to the Lineup
    for _, skater in skaters.iterrows():
        try:
            position_enum = Position(skater['position'])  # Convert to Position Enum
        except ValueError:
            print(f"Invalid position '{skater['position']}' for player '{skater['name']}'. Skipping.")
            continue

        player = Player(
            name=skater['name'],
            team=team.upper(),
            position=position_enum
        )

        # Add player to the appropriate category in the lineup
        if player.position.category == 'F':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
                lineup.add_forward(player, empty_slot)
            except StopIteration:
                print(f"No available forward slot to add player '{player.name}'.")
        elif player.position.category == 'D':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.defense) if p is None)
                lineup.add_defense(player, empty_slot)
            except StopIteration:
                print(f"No available defense slot to add player '{player.name}'.")
        else:
            print(f"Player '{player.name}' has an unrecognized category '{player.position.category}'. Skipping.")

    # Add Goalies to the Lineup
    for _, goalie in goalies.iterrows():
        player = Player(
            name=goalie['name'],
            team=team.upper(),
            position=Position.G
        )
        try:
            empty_slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, empty_slot)
        except StopIteration:
            print(f"No available goalie slot to add player '{player.name}'.")

    return lineup

In [65]:
toronto_lineup = extract_team_lineup('TOR')
toronto_lineup.to_dataframe()

Added player 'M. Domi' to Forward slot 1.
Added player 'M. Marner' to Forward slot 2.
Added player 'S. Lorentz' to Forward slot 3.
Added player 'M. Knies' to Forward slot 4.
Added player 'C. Dewar' to Forward slot 5.
Added player 'P. Holmberg' to Forward slot 6.
Added player 'D. Kampf' to Forward slot 7.
Added player 'B. McMann' to Forward slot 8.
Added player 'R. Reaves' to Forward slot 9.
Added player 'W. Nylander' to Forward slot 10.
Added player 'N. Robertson' to Forward slot 11.
Added player 'J. Tavares' to Forward slot 12.
Added player 'C. Tanev' to Defens slot 1.
Added player 'J. McCabe' to Defens slot 2.
Added player 'C. Timmins' to Defens slot 3.
Added player 'J. Hakanpää' to Defens slot 4.
Added player 'M. Rielly' to Defens slot 5.
Added player 'O. Ekman-Larsson' to Defens slot 6.
Added player 'A. Stolarz' to Goalie slot 1.
Added player 'J. Woll' to Goalie slot 2.


,Position,Player
0,f1,M. Domi
1,f2,M. Marner
2,f3,S. Lorentz
3,f4,M. Knies
4,f5,C. Dewar
5,f6,P. Holmberg
6,f7,D. Kampf
7,f8,B. McMann
8,f9,R. Reaves
9,f10,W. Nylander
